Including one-hot encoded categorigal data along with numerical data, when using a DecisionTreeClassifier.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')


# Any results you write to the current directory are saved as output.

Bring in as panda dataframes
* the labels
* the train data as orig_train_df (because we will split them into *training* and *validation* data
* the test data

Combine the original training and testing data into one dataframe, so that the transformations we make apply to all of them. (First mark all the original training and test data as train and test accordingly so we can separate them later.)

In [ ]:
breeds_df = pd.read_csv('../input/breed_labels.csv')

orig_train_df = pd.read_csv('../input/train/train.csv')
test_df = pd.read_csv('../input/test/test.csv')

orig_train_df['dataset_type'] = 'train'
test_df['dataset_type'] = 'test'
all_data_df = pd.concat([orig_train_df, test_df])

print ("all_data_df shape: ", all_data_df.shape)
print ("orig_train_df shape: ", orig_train_df.shape)
print ("test_df shape: ", test_df.shape)


Before manipulating any data, let's extract the series (column) of the PetID from the test data.  Toward the end of this kernel, we will use the sample submission file to submit the results.  Before submitting, we can verify that the order of PetIDs for which we have results is the same as the order of PetIDs in the sample submission file.  (This will make more sense when we get to that point; for now we only have to obtain that series.)

In [ ]:
print (list(test_df.columns))
the_test_df_pet_ids = test_df["PetID"]
print(the_test_df_pet_ids.head())

We will need 
* a list of the numerical fields we will use
* a list of the categorical fields we will use, and 
* a list of the categorical fields we will not use (so we can drop them from the dataframe prior to training the model.)

In [ ]:
list_of_num_fields = [ 'Age', 'MaturitySize', 'Quantity', 'Fee', 'VideoAmt', 'PhotoAmt', ]
list_of_categ_fields = ['Breed1']
list_to_drop = [ "RescuerID", "Name", "Description", "PetID", 'dataset_type']


# Explore the Breeds1 data

The Breeds1 column in the all_data dataframe holds a *numerical* value representing Breed1, rather than holding the breed name in *text*.

To see the breed name in text, we can use the lookup dictionary which we already imported into breeds_df. The length of that dataframe is 307 rows, yet the last row (Tuxedo cat) has the numerical  value 306. That's because 307 stands for Mixed Breed and is tucked between the alphabetically-listed dog breeds, and all the cat breeds.

Important data science: I used to have a Category 304 cat and she was the love of my life.  Smartest person I ever met 🤔😻



In [ ]:
print("What the head() of Breeds1 column looks like in the all_data dataframe :",all_data_df["Breed1"].head())

print("\nLength of breeds_df.BreedID: ", len(breeds_df.BreedID))
print("\nWhat the head() Breeds1 lookup table looks like:\n", breeds_df.head())
print("\nWhat the tail() Breeds1 lookup table looks like:\n", breeds_df.tail())
print("\nThe numerical values are of type: ", type(breeds_df.BreedID[3]))

if 307 in list(breeds_df.BreedID):
    print ("Category 307 (Mixed Breed) is in the dictionary, at position: " , 
           list(breeds_df.BreedID).index(307), 
           ". It is wedged between all the dog breeds and all the cat breeds:") 
    print (breeds_df[238:243])

# One-Hot Encode the Chosen Columns

Breed1 is the only column we are one-hot encoding.

Testing out how to get 307 columns for one-hot encoding using get_dummies, for the head of the all_data_df.Breed1 column.

In [ ]:
the_one_hot_encodings = pd.get_dummies(all_data_df.Breed1.head(5), prefix = 'Breed1')
print (the_one_hot_encodings.head(5))
print("\nThe columns are: ", the_one_hot_encodings.columns)

Because only at most five categories would be used in the first five rows, we get new columns for only those categories. For example, the first time I run the above, there were only 3 columns created and the .columns statement lists them as 265, 299, and 307:

```Index(['Breed1_265', 'Breed1_299', 'Breed1_307'], dtype='object')```

Using the above method we run the risk of missing columns, if any categories do not exist in the training data (which we will extract again shortly) but do show up in the given test data or future test data.

A better way to handle this would be to provide a complete list of potential categories to the dataframe, so that when creating the dummies all possible columns would be created.

In [ ]:
all_data_df["Breed1"] = all_data_df["Breed1"].astype('category', categories = list(breeds_df.BreedID) )
the_one_hot_encodings = pd.get_dummies(all_data_df.Breed1.head(), prefix = 'Breed1')
print(the_one_hot_encodings.head(5))
print(the_one_hot_encodings.columns)

Concatenate, as columns, the two dataframes to get one wider dataframe, then split back to the original training data and test data.

In [ ]:
all_data_df = pd.concat( [all_data_df, pd.get_dummies(all_data_df.Breed1, prefix = 'Breed1') ] , axis = 1)
print("all_data_df shape: ", all_data_df.shape)
print("Now extract from that the following:")
#Split them back into the original train vs. test data
orig_train_df = all_data_df[all_data_df.dataset_type == "train"]
test_df = all_data_df[all_data_df.dataset_type == "test"]
print("orig_train_df shape:", orig_train_df.shape)
print("test_df shape:", test_df.shape)

Furthermore, split the original training data into training data and validation data, at 80-20 split.
sklearn provides an easy way of doing this

In [ ]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(orig_train_df, test_size = .2)

print ("train_df shape:", train_df.shape)
print ("val_df shape:", val_df.shape)

Drop the unneeded columns

In [ ]:
train_targets = train_df["AdoptionSpeed"]
train_df.drop( ["AdoptionSpeed"] + list_to_drop + ["Breed1"], axis =1, inplace = True)
val_targets = val_df["AdoptionSpeed"]
val_df.drop(["AdoptionSpeed"] + list_to_drop + ["Breed1"], axis = 1, inplace = True)

test_df.drop(["AdoptionSpeed"] + list_to_drop + ["Breed1"], axis = 1, inplace = True)

train_df.isna().sum()

In [ ]:
print ("\tall_data_df has been: ", all_data_df.shape)
print ("From which we extracted :")
print ("\torig_train_df", orig_train_df.shape)
print ("\ttest_df", test_df.shape)
print ("\nAnd furthermore, we split the orig_train_df into:")
print ("\ttrain_df", train_df.shape)
print ("\tval_df", val_df.shape)

# Create and Train the Model

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import cohen_kappa_score

clf = DecisionTreeClassifier(random_state=0, max_leaf_nodes=22)

Train the model

In [ ]:
clf.fit(train_df, train_targets)

Make a single prediction.
(Note that the prediction comes out as a *float*, which we need to remember when later on we submit the results.)

In [ ]:
clf.predict(test_df.iloc[3:4])

Make predictions on the validation set, and calculate the CKS

In [ ]:
val_preds = clf.predict(val_df)
cohen_kappa_score(val_preds, val_targets, weights = "quadratic")


# Prepare and Submit the Results

In [ ]:
test_preds = clf.predict(test_df)

Create a sub_df dataframe to hold the submission of the results, by reading the sample_submission.csv .
Ensure that the order of pets in  prediction set is the same as the order of pets in the sub_df, and if so, simply replace the "AdoptionSpeed" of the sub_df with our prediction results - but convert the floats to integers first.

In [ ]:
sub_df = pd.read_csv('../input/test/sample_submission.csv')
print(sub_df.PetID.head())
print()
print(the_test_df_pet_ids.head())

print("Are the two series types comparable:", type(sub_df.PetID) == type(the_test_df_pet_ids))
print("The two series are identical: ", sub_df["PetID"].equals(the_test_df_pet_ids))

sub_df['AdoptionSpeed'] = test_preds.astype(int)

Convert the _df to a csv file and write it.
(After writing it we have to manually submit it, outside of this kernel.)

In [ ]:
sub_df.to_csv('submission.csv', index=False)

# Visually Inspect the Decision Tree

In [ ]:
from sklearn import tree
import graphviz

outfile = 'ourtree.dot'
dot_data = tree.export_graphviz(clf, out_file=outfile,
                                feature_names=train_df.columns,  
                                class_names=['0','1','2','3','4'],  
                                filled=True, rounded=True, max_depth=4, 
                                special_characters=True) 

In [ ]:
#Efforts to create a png
#!ls
#!rm ourtree.png
#!ls
#!dot -Tpng ourtree.dot -o ourtree.png
#!ls
#%matplotlib inline
#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
#img = mpimg.imread('ourtree.png')
#plt.imshow(img)

In [ ]:
dot_data = tree.export_graphviz(clf, out_file=None,
                                feature_names=train_df.columns,  
                                class_names=['0','1','2','3','4'],  
                                filled=True, rounded=True, max_depth=4, 
                                special_characters=True) 

graph = graphviz.Source(dot_data)  # you can't have these two lines when trying out_file (for some reason)
graph 